An example Scholia / Wikidata / WikiCite profile for "environmental justice" is here
https://scholia.toolforge.org/topic/Q1479527

for the resource hub, we may want ["Recently published works on the topic"]([text](https://scholia.toolforge.org/topic/Q1479527#recently-published-works)), as with https://scholia.toolforge.org/topic/Q1479527#recently-published-works

The SPARQL query for that is
https://github.com/WDscholia/scholia/blob/master/scholia/app/templates/topic_recently-published-works.sparql

In [27]:
query = '''
PREFIX target: <http://www.wikidata.org/entity/Q1479527>

SELECT ?date ?work ?workLabel (CONCAT("/work/", SUBSTR(STR(?work), 32)) AS ?workUrl)
  ?topicsUrl ?topics 
WITH {
  SELECT DISTINCT ?work WHERE {
    ?work wdt:P921 / (wdt:P361+ | wdt:P1269+ | (wdt:P31* / wdt:P279*) ) target: .
  }
} AS %works
WITH {
  SELECT (MAX(?dates) as ?datetime) ?work (GROUP_CONCAT(DISTINCT ?topic_label; separator=" // ") AS ?topics) 
                                    (CONCAT("../topics/", GROUP_CONCAT(DISTINCT SUBSTR(STR(?topic), 32); separator=",")) AS ?topicsUrl) 
  WHERE {
    INCLUDE %works
    ?work wdt:P921 ?topic . 
    OPTIONAL { ?work wdt:P577 ?dates . }
    ?topic rdfs:label ?topic_label .  FILTER (lang(?topic_label) = 'en')
  }
  GROUP BY ?work
} AS %result
WHERE {
  INCLUDE %result

  # There is a problem with BC dates
  # BIND(xsd:date(?datetime) AS ?date)
  BIND(REPLACE(STR(?datetime), 'T.*', '') AS ?date)
    
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,nl,no,ru,sv,zh". }
}
GROUP BY ?date ?work ?workLabel ?topicsUrl ?topics
ORDER BY DESC(?date)
LIMIT 500
'''

In [28]:
import python.data_extraction as DEX

data_extracter = DEX.WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()
df.head()

,work,workLabel,topicsUrl,topics,workUrl,date
0,http://www.wikidata.org/entity/Q123555983,Climate Equality: A planet for the 99%,../topics/Q1291678,climate justice,/work/Q123555983,2023-11-20
1,http://www.wikidata.org/entity/Q123194943,Pursuit of environmental justice in urban fore...,../topics/Q1479527,environmental justice,/work/Q123194943,2023-08-01
2,http://www.wikidata.org/entity/Q123017569,Evaluating the quality of street trees in Wash...,../topics/Q1479527,environmental justice,/work/Q123017569,2023-07-01
3,http://www.wikidata.org/entity/Q122782904,Patterns of contamination and burden of lead a...,"../topics/Q110315,Q1479527,Q2137507,Q730249",lead poisoning // environmental justice // rai...,/work/Q122782904,2023-07-01
4,http://www.wikidata.org/entity/Q123615639,Compensation for atmospheric appropriation,"../topics/Q1291678,Q16722080",climate justice // carbon budget,/work/Q123615639,2023-06-05
